In [106]:
%load_ext autoreload
%autoreload 2
import time
from tweepy import TweepError
from requests.exceptions import Timeout, ConnectionError
import os
import tweepy
import json
import requests
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# THE CELL BELOW CONTAINS THE FILENAME CONSTANTS THAT NEED TO BE CHANGED

In [107]:
'''
    SOME QUICK HOW TO!
    
    T
'''



# This file should be the one with the tweets as jsons in it.
TWEET_ID_FILE = "C:/Users/mfren/Desktop/test.json"

# The downloaded images should go into a folder.
OUTPUT_DIR = "C:/Users/mfren/Desktop/test_images/"

In [108]:
# setup tweepy
consumer_key = "PQSsqSxGft69D53XGknXya8cL"
consumer_secret = "dlh6EFLADanVNvdl9Q9Lz9Iv3yQQyHY9vxKTS9BN2skflGd45d"
access_token = "1187382763432771584-rwZbZc9lcOcaVdgxcIgqqCPqoQMo3I"
access_token_secret = "nvnIFiZD9L1u6tZwntl24BtsLhKmkBntJ4dpko82fYKg8"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True) # this param makes it not have issues with API limits


In [109]:
def exists(path):
  r = requests.head(path)
  return r.status_code == requests.codes.ok

In [110]:
with open(TWEET_ID_FILE) as f:
    all_content_as_string = f.read()

In [111]:
# split the file into a list of strings.
list_of_dicts = all_content_as_string.splitlines()

# turn each string into a json object
list_of_jsons = [json.loads(string) for string in list_of_dicts]

In [112]:
# Go through the whole list of tweets and download the images.

for i in range(0, len(list_of_jsons)):
    temp = dict(list_of_jsons[i])
    url = temp['entities']['media'][0]['media_url_https']

    if(exists(url)):
        url_raw = requests.get(url, stream=True)
        url_raw.raw.decode_content = True
        
        with open(OUTPUT_DIR + str(temp['id']) + '_' + str(temp['user']['id']) + '_' + str(i) + ".jpg", 'wb') as fin:
            shutil.copyfileobj(url_raw.raw, fin)
        
        # Remove the image url response object - because you should.
        del url_raw
    else:
        continue
        
# So you know once it is done.
print("DONE!")